In [1]:
pip install scholarly

     |████████████████████████████████| 2.9 MB 10.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 2.8 MB 55.3 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 954 kB 53.5 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 3.5 MB 38.9 MB/s 
     |████████████████████████████████| 356 kB 32.3 MB/s 
     |████████████████████████████████| 138 kB 58.9 MB/s 
     |████████████████████████████████| 138 kB 59.6 MB/s 
     |████████████████████████████████| 138 kB 65.8 MB/s 
     |████████████████████████████████| 153 kB 68.9 MB/s 
     |████████████████████████████████| 137 kB 54.7 MB/s 
     |████████████████████████████████| 136 kB 58.1 MB/s 
     |████████████████████████████████| 136 kB 57.6 MB/s 
     |████████████████████████████████| 136 kB 56.9 MB/s 
INFO: pip is looking at multiple versions of trio-websocket to determine which

In [2]:
from scholarly import scholarly
import pandas as pd

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [3]:
df1 = pd.read_excel('/content/CSED.xlsx')
df2 = pd.read_excel('/content/CSED_derabassi.xlsx')
df3 = pd.read_excel('/content/Chemical.xlsx')
df4 = pd.read_excel('/content/Electrical & Instrumentation.xlsx')
df5 = pd.read_excel('/content/Electronics & Communication.xlsx')
df6 = pd.read_excel('/content/Mechanical.xlsx')
df7 = pd.read_excel('/content/biotech.xlsx')
df8 = pd.read_excel('/content/civil.xlsx')
df9 = pd.read_excel('/content/distant_edu.xlsx')

In [4]:
import re
l = [df1, df2, df3, df4, df5, df6, df7, df8, df9]
prefixes = ['Mr', 'Dr', 'Mrs','Ms']
regex = r'\b(?:' + '|'.join(prefixes) + ')\.\s*'
def name_update(j):
  j = re.sub(regex,'',j)
  return j

for i in l:
  i['Name'] = i['Name'].apply(lambda x: name_update(x))
print(l)


[                Name  ...                         faculty-acad 4
0       R. K. Sharma  ...                    rksharma@thapar.edu
1         Seema Bawa  ...                       seema@thapar.edu
2       Rajesh Kumar  ...                     rakumar@thapar.edu
3    Inderveer Chana  ...  inderveer@thapar.edu, dosa@thapar.edu
4     Maninder Singh  ...                      msingh@thapar.edu
..               ...  ...                                    ...
86       Sanjeev Rao  ...                 sanjeev.rao@thapar.edu
87        Tanya Garg  ...                  tanya.garg@thapar.edu
88       Amrita Kaur  ...                      amrita@thapar.edu
89      Chinu Singla  ...                 chinusingla@thapar.edu
90  Harmandeep Singh  ...            harmandeep.singh@thapar.edu

[91 rows x 4 columns],               Name  ...              faculty-acad 4
0         AK Verma  ...           akvera@thapar.edu
1  Manisha Kaushal  ...  manisha.kaushal@thapar.edu
2    Abhishek Jain  ...    abhishek.jai

In [5]:
d = {}
l = [df1, df2, df3, df4, df5, df6, df7, df8, df9]

x = 1
for i in l:
  d["dept{0}".format(x)] = i['Name']
  x = x+1
print(d)

{'dept1': 0         R. K. Sharma
1           Seema Bawa
2         Rajesh Kumar
3      Inderveer Chana
4       Maninder Singh
            ...       
86         Sanjeev Rao
87          Tanya Garg
88         Amrita Kaur
89        Chinu Singla
90    Harmandeep Singh
Name: Name, Length: 91, dtype: object, 'dept2': 0           AK Verma
1    Manisha Kaushal
2      Abhishek Jain
3        Vivek Gupta
4          Deep Mann
Name: Name, dtype: object, 'dept3': 0               Rajeev Mehta
1     Dasaroju Gangacharyulu
2            Haripada Bhunia
3            Raj Kumar Gupta
4         Sanghamitra Barman
5        Sanjeev Kumar Ahuja
6       Jai Prakash Kushwaha
7            Avinash Chandra
8         Sudhir Kumar Singh
9                Neetu Singh
10           Parminder Singh
11        Rakesh Kumar Gupta
Name: Name, dtype: object, 'dept4': 0                  R. S. Kaler
1               Smarajit Ghosh
2             Ravinder Agarwal
3                Mandeep Singh
4                  Sanjay Jain
5        

In [6]:
def checkKey(dict, key): 
    if key in dict.keys():
        return 1
    else:
        return 0

In [7]:
def main_func(y):
  p = 0
  ## for loop outside main code:
  for k in d:
    total_count = []
    for i in d[k]:
      try:
        search_query = scholarly.search_author(i)
        author = scholarly.fill(next(search_query))
        author = author['publications']
      except StopIteration:
        pass

      papers_count = {"Title":[],"year":[]};
      for i in range(len(author)):
        if checkKey(author[i]['bib'], 'pub_year'):
          if checkKey(author[i]['bib'], 'title'):
            papers_count["Title"].append(author[i]['bib']['title'])
            papers_count["year"].append(author[i]['bib']['pub_year'])

      #print(papers_count)

      ## Datafame of the publications and dates of a faculity member: 
      df = pd.DataFrame.from_dict(papers_count, orient='index')
      df = df.transpose()
      # print(df)
      df = df.sort_values(by = 'year', ascending = False)
      # print(df)

      #count for yearly publications:
      i = 0
      a = [i+1 if x == y else i for x in df.year]
      # print(a)
      count = 0
      for i in a:
        if i>0:
          count = count+1
      print(count)
      total_count.append(count)
    m = l[p]
    m['count'] = total_count
    m.to_csv('/content/sample_data/modified_data/'+str(p)+ '_' + y)
    p = p + 1
    print(total_count)
    print(m)
      

In [8]:
y = ['2018','2019','2020','2021']
for yr in y:
  main_func(yr)

14
9
233
10
19
5
25
15
27
20
13
118
13
255
12
6
9
8
154
9
2
2
4
7
4
4
4
4
3
3
21
4


'geckodriver' executable needs to be in PATH. 
Fetching only the top 20 coauthors


18
15
2
2
6
7
0
1
9
4
4
0
8
23
0
5
2
2
6
12
0
4
1
135
4
2
3
17
211
8
0
2
6
1
1
0
5
0
1
0
68
0
0
41
41
3
2
2
154
0
22
3
3
3
0
0
3
3
14
[14, 9, 233, 10, 19, 5, 25, 15, 27, 20, 13, 118, 13, 255, 12, 6, 9, 8, 154, 9, 2, 2, 4, 7, 4, 4, 4, 4, 3, 3, 21, 4, 18, 15, 2, 2, 6, 7, 0, 1, 9, 4, 4, 0, 8, 23, 0, 5, 2, 2, 6, 12, 0, 4, 1, 135, 4, 2, 3, 17, 211, 8, 0, 2, 6, 1, 1, 0, 5, 0, 1, 0, 68, 0, 0, 41, 41, 3, 2, 2, 154, 0, 22, 3, 3, 3, 0, 0, 3, 3, 14]
                Name  ... count
0       R. K. Sharma  ...    14
1         Seema Bawa  ...     9
2       Rajesh Kumar  ...   233
3    Inderveer Chana  ...    10
4     Maninder Singh  ...    19
..               ...  ...   ...
86       Sanjeev Rao  ...     0
87        Tanya Garg  ...     0
88       Amrita Kaur  ...     3
89      Chinu Singla  ...     3
90  Harmandeep Singh  ...    14

[91 rows x 5 columns]


'geckodriver' executable needs to be in PATH. 
Fetching only the top 20 coauthors


43
10


...

18
17
27
2
1
1
0
4
3
1
0
2
0
1
7
1
[2, 2, 3, 0, 1, 1, 60, 3, 9, 9, 0, 1, 2, 7, 3, 9, 9, 9, 4, 0, 2, 9, 2, 1, 3, 5, 1, 0, 0, 2, 2, 3, 4, 0, 12, 9, 1, 0, 0, 0, 18, 17, 27, 2, 1, 1, 0, 4, 3, 1, 0, 2, 0, 1, 7, 1]
                           Name  ... count
0               S. K. Mohapatra  ...     2
1         Professor Ajay Batish  ...     2
2              Tarun Kumar Bera  ...     3
3                 A. S. Jawanda  ...     0
4                 Sumeet Sharma  ...     1
5               Supreet Bhullar  ...     1
6                   Vinod Kumar  ...    60
7                   S.S.Mallick  ...     3
8                   Tarun Nanda  ...     9
9         Jaswinder Singh Saini  ...     9
10                   Vivek Jain  ...     0
11                Dheeraj Gupta  ...     1
12         Sandeep Kumar Sharma  ...     2
13                Ashish Singla  ...     7
14          Madhup Kumar Mittal  ...     3
15            Anant Kumar Singh  ...     9
16          Ravinder Kr. Duvedi  ...     9
17             Hi

'geckodriver' executable needs to be in PATH. 
Fetching only the top 20 coauthors


8
15
1
11
[11, 6, 4, 6, 1, 8, 10, 2, 0, 2, 2, 2, 8, 15, 1, 11]
                   Name  ... count
0             M S Reddy  ...    11
1         Das, N. (Dr,)  ...     6
2          Dinesh Goyal  ...     4
3         Sanjai Saxena  ...     6
4               M Ghosh  ...     1
5            Anil Kumar  ...     8
6        Manoj Baranwal  ...    10
7      Siddharth Sharma  ...     2
8           Vikas Handa  ...     0
9         M. Vasundhara  ...     2
10  Atul Kumar Upadhyay  ...     2
11   Ovais Shafiq Qadri  ...     2
12        Priyankar Dey  ...     8
13      Debajyoti Dutta  ...    15
14          Nishu Joshi  ...     1
15  Prangya Ranjan Rout  ...    11

[16 rows x 5 columns]
11
12
12
0
21
8
1
5
0
1
1
0
1
1
2
0
2
1
1
0
1
77
127
0
4
[11, 12, 12, 0, 21, 8, 1, 5, 0, 1, 1, 0, 1, 1, 2, 0, 2, 1, 1, 0, 1, 77, 127, 0, 4]
                  Name  ... count
0      Prem Pal Bansal  ...    11
1       Rafat Siddique  ...    12
2        Naveen Kwatra  ...    12
3        Sarbjit Singh  ...     0
4        

'geckodriver' executable needs to be in PATH. 
Fetching only the top 20 coauthors


0
[0]
            Name     int-desc  ...         int-desc 3 count
0  Rajesh Khanna  Coordinator  ...  rkhanna@thapr.edu     0

[1 rows x 5 columns]
